In [4]:
import torch
from PIL import Image
import clip
from transformers import SiglipProcessor, SiglipModel
import torch.nn.functional as F

# ====== 1. 加载本地模型 ======
# 替换为你本地的路径
clip_model, clip_preprocess = clip.load("ViT-B/32", device="cuda" if torch.cuda.is_available() else "cpu")

siglip_model_path = "./siglip_local"  # ← 你的本地siglip路径
siglip_model = SiglipModel.from_pretrained(siglip_model_path).eval()
siglip_processor = SiglipProcessor.from_pretrained(siglip_model_path)

device = "cuda" if torch.cuda.is_available() else "cpu"
siglip_model = siglip_model.to(device)

# ====== 2. 加载图像和文本 ======
image_path = "roses.jpg"  # 你自己的图片路径
text = "Find white roses in the image. Paint the white rose into red"  # 你自己的文本描述

# CLIP 预处理
clip_image = clip_preprocess(Image.open(image_path)).unsqueeze(0).to(device)
clip_text = clip.tokenize([text]).to(device)

# SigLIP 预处理
siglip_inputs = siglip_processor(images=Image.open(image_path), text=[text], return_tensors="pt").to(device)

# ====== 3. 前向传播 ======
with torch.no_grad():
    # CLIP 输出
    clip_img_emb = clip_model.encode_image(clip_image)  # (1, D)
    clip_txt_emb = clip_model.encode_text(clip_text)    # (1, D)

    # SigLIP 输出
    siglip_outputs = siglip_model(**siglip_inputs)
    siglip_img_emb = siglip_outputs.image_embeds        # (1, D)
    siglip_txt_emb = siglip_outputs.text_embeds         # (1, D)

# ====== 4. 标准化和对比 ======
def cosine_sim(a, b):
    return F.cosine_similarity(a, b).item()

clip_img_emb_norm = F.normalize(clip_img_emb, dim=-1)
clip_txt_emb_norm = F.normalize(clip_txt_emb, dim=-1)

siglip_img_emb_norm = F.normalize(siglip_img_emb, dim=-1)
siglip_txt_emb_norm = F.normalize(siglip_txt_emb, dim=-1)

print("\n===== CLIP 输出 =====")
print("图像 embedding:", clip_img_emb.squeeze().tolist()[:5], "...")
print("文本 embedding:", clip_txt_emb.squeeze().tolist()[:5], "...")
print("相似度:", cosine_sim(clip_img_emb_norm, clip_txt_emb_norm))
# 输出embedding维度
print("图像 embedding 维度:", clip_img_emb.shape)
print("文本 embedding 维度:", clip_txt_emb.shape)

print("\n===== SigLIP 输出 =====")
print("图像 embedding:", siglip_img_emb.squeeze().tolist()[:5], "...")
print("文本 embedding:", siglip_txt_emb.squeeze().tolist()[:5], "...")
print("相似度:", cosine_sim(siglip_img_emb_norm, siglip_txt_emb_norm))
# 输出embedding维度
print("图像 embedding 维度:", siglip_img_emb.shape)
print("文本 embedding 维度:", siglip_txt_emb.shape)



===== CLIP 输出 =====
图像 embedding: [0.198974609375, 0.46337890625, -0.146240234375, -0.146240234375, -0.11224365234375] ...
文本 embedding: [-0.038299560546875, -0.050872802734375, 0.248046875, 0.382568359375, 0.003879547119140625] ...
相似度: 0.271484375
图像 embedding 维度: torch.Size([1, 512])
文本 embedding 维度: torch.Size([1, 512])

===== SigLIP 输出 =====
图像 embedding: [-0.025411736220121384, -0.0158709529787302, -0.02022467739880085, 0.016594942659139633, 0.03929762914776802] ...
文本 embedding: [-0.05442783236503601, -0.029097648337483406, 0.027545589953660965, -0.010813655331730843, 7.1288290200755e-05] ...
相似度: -0.013044867664575577
图像 embedding 维度: torch.Size([1, 768])
文本 embedding 维度: torch.Size([1, 768])
